##### Importing Packages

In [267]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sn
import numpy as np

##### Loading Data

In [10]:
TASK_PATH = r"C:\Users\jpgsa\Documents\BOLT" + "\\"
file1 = "Dataset Task 1.xlsx"
df1_raw = pd.read_excel(TASK_PATH + file1)

df1_raw.head()

,Time,Category level 0,Category level 1,Category level 2,Unique SKUs Listed,Unique SKUs Sold,# of sold SKU items,Price before Discount (includ. VAT) (EUR),Discount Value (EUR),Item COGS (net VAT) (EUR),"Waste, # of items"
0,2023-09-17,Fresh Food,🧀 Cheese,Hard Cheese,26,17,37,135.812592,10.491467,88.81738,0
1,2023-08-02,Fresh Food,🍞 Bakery,Gluten Free,1,0,0,0.000000,0.000000,0.00000,0
2,2023-08-26,Fresh Food,🥛 Dairy & Eggs,Sour Cream & Cream,14,10,26,53.454514,3.213313,31.87778,0
3,2023-09-24,Fresh Food,🥪 Fresh & Ready,Desserts & Sweets,25,13,47,61.698558,3.516517,41.22566,0
4,2023-09-15,Fresh Food,🍗 Meat & Fish,Meat & Fish Alternatives,17,3,8,13.061563,1.358066,8.68154,0


In [85]:
df1 = df1_raw.copy()
df1["perc_discount"] = df1["Discount Value (EUR)"] / df1["Price before Discount (includ. VAT) (EUR)"]
df1["category_tree"] = df1["Category level 0"] + "/" +  df1["Category level 1"] + "/" + df1["Category level 2"]
df1["avg_item_price"] = df1["Price before Discount (includ. VAT) (EUR)"] / df1["# of sold SKU items"]

##### 1) Discount Elasticity - Correlation between **qty itens** and **% discount** daily, per item

In [130]:
df_elast1 = df1.groupby('category_tree')[['perc_discount','# of sold SKU items']].corr().iloc[0::2,-1].reset_index().drop("level_1", axis = 1).rename(columns={"# of sold SKU items": "correl_disc_and_qty"})

df_temp = df1[["category_tree", "# of sold SKU items","avg_item_price"]]
df_temp = df_temp.groupby("category_tree").mean()[["# of sold SKU items","avg_item_price"]].reset_index().rename(columns={"# of sold SKU items": "avg_itens_sold_daily"})

df_elast2 = pd.merge(df_elast1, df_temp, left_on='category_tree', right_on='category_tree')
df_elast2.sort_values("correl_disc_and_qty", ascending=False, inplace=True)
df_elast2.reset_index(drop=True, inplace=True)

df_elast2.head(20)

,category_tree,correl_disc_and_qty,avg_itens_sold_daily,avg_item_price
0,Grocery Food/🥤 Beverages/More Beverages,0.935812,0.120879,3.558840
1,Fresh Food/🍗 Meat & Fish/Fresh Meat,0.503441,37.131868,4.476076
2,Fresh Food/🍗 Meat & Fish/Fresh Poultry,0.470720,40.945055,4.326510
3,Fresh Food/🥕 Fruits & Vegetables/Berries,0.454022,48.000000,2.040084
4,Grocery Food/❄️ Frozen Products/Frozen Fruits ...,0.380585,4.923077,5.278142
5,Grocery Food/🧑‍🍳 Pantry/Sugar Free Pantry,0.371779,0.252747,2.827747
6,Grocery Food/🍦 Ice Cream/Large,0.368116,69.670330,8.084641
7,Grocery Food/🥃 Spirits/Liqueurs,0.366234,1.340659,19.181716
8,Grocery Food/❄️ Frozen Products/Frozen Meals,0.341998,19.395604,3.996358
9,Grocery Food/🥃 Spirits/Vodka,0.341005,5.648352,12.427498


##### 2) Itens Afinity - Correlation of itens sold daily, between different itens

In [323]:
df_temp = df1[["Time","category_tree", "# of sold SKU items"]]
df_temp_pivot = df_temp.pivot(index='Time', columns='category_tree', values='# of sold SKU items')
corr_matrix  = df_temp_pivot.corr()
corr_matrix.rename(columns={"category_tree": "category_tree_2"})

corr_pairs = corr_matrix.unstack()
corr_pairs.index.names = ["Category A", "Category B"]
corr_pairs = corr_pairs.reset_index()
corr_pairs = corr_pairs.rename(columns={0: "Correl"})
corr_pairs = corr_pairs[corr_pairs["Category A"] != corr_pairs["Category B"]]

corr_pairs['Pair'] = np.where(corr_pairs['Category A'] > corr_pairs['Category B'] 
                              , corr_pairs['Category A'] + " + " + corr_pairs['Category B']
                              , corr_pairs['Category B'] + " + " + corr_pairs['Category A'])

corr_pairs = corr_pairs[["Pair","Correl"]]
corr_pairs.sort_values("Correl", ascending=False, inplace=True)
corr_pairs =corr_pairs.drop_duplicates(subset=["Pair"])
corr_pairs.reset_index(inplace=True, drop=True)

corr_pairs.head(20)
#for index, row in corr_pairs.head(20).iterrows():
#    print("Correlation: ",row['Correl'], "  Combo: ",row['Pair'].split(" + "))

,Pair,Correl
0,Grocery Food/🥤 Beverages/Kombucha + Fresh Food...,0.607488
1,"Fresh Food/🍞 Bakery/Sliced Bread, Wraps & More...",0.568021
2,Grocery Food/🍷 Wine/White Wine + Grocery Food/...,0.567165
3,Fresh Food/🥕 Fruits & Vegetables/Vegetables + ...,0.560113
4,Fresh Food/🥛 Dairy & Eggs/Yogurt & Pudding + F...,0.556547
5,Grocery Food/🍺 Beer & Cider/International Beer...,0.548642
6,Grocery Food/🍺 Beer & Cider/National Favourite...,0.540685
7,Grocery Food/🍬 Sweet Snacks/Biscuits + Fresh F...,0.538862
8,Other/♻️ Save me!/Packaging fee + Grocery Food...,0.536971
9,Other/♻️ Save me!/Packaging fee + Grocery Food...,0.534980


##### 3) Catalog Size vs. % Catalog Sold Daily

##### 4) Estimated Loss Daily (based on wasted items avg. COGS)